In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os

# Set the size of the 3D structure
slice_size = (100, 100)  # Each slice is 100x100 pixels
num_slices = 100         # Number of slices
num_structures_per_case = 800  # Number of structures for each case

# Pore sizes and their corresponding names
pore_sizes = [10]
density_labels = ['dataset']

# Number of pores for each density
density_pores = {
    'dataset': (10, 40),    # Low density: 3-5 pores
    #'medium_density': (6, 10), # Medium density: 6-10 pores
    #'high_density': (11, 16)   # High density: 11-16 pores
}

# Directory to save the structures
output_folder = "3D_square"
os.makedirs(output_folder, exist_ok=True)

# Function to check if a rectangular pore overlaps with existing pores
def is_overlapping(existing_mask, start_row, start_col, pore_height, pore_width):
    # Check if the area where the pore would be placed is already occupied (non-zero)
    return np.any(existing_mask[start_row:start_row + pore_height, start_col:start_col + pore_width] == 0)

# Function to place multiple non-overlapping rectangular pores in 3D structure
def generate_rectangular_pores_3D(pore_height, pore_width, num_pores):
    # Create a 3D array (num_slices x height x width) filled with ones (white background, representing material)
    structure = np.ones((num_slices, slice_size[0], slice_size[1]))

    for _ in range(num_pores):
        pore_placed = False
        while not pore_placed:
            # Randomly position the pore in the slice
            start_row = np.random.randint(0, slice_size[0] - pore_height)
            start_col = np.random.randint(0, slice_size[1] - pore_width)

            # Check for overlap with existing pores
            if not is_overlapping(structure[0], start_row, start_col, pore_height, pore_width):
                # If there's no overlap, place the same pore in all slices (continuous pipe)
                structure[:, start_row:start_row + pore_height, start_col:start_col + pore_width] = 0  # Pore is black (0)
                pore_placed = True

    return structure

# Generate and save 3D structures
for pore_size in pore_sizes:
    for density_label in density_labels:
        # Get the number of pores based on the density
        min_pores, max_pores = density_pores[density_label]

        # Create a folder for this pore size and density distribution
        case_folder = os.path.join(output_folder, f"pore_{pore_size}x{pore_size}", density_label)
        os.makedirs(case_folder, exist_ok=True)

        for structure_idx in range(num_structures_per_case):
            # Randomly generate the number of pores for this structure
            num_pores = np.random.randint(min_pores, max_pores + 1)

            # Generate the 3D structure with non-overlapping rectangular pores
            structure = generate_rectangular_pores_3D(pore_size, pore_size, num_pores)

            # Create a folder for each 3D structure
            structure_folder = os.path.join(case_folder, f"structure_{structure_idx:05d}")
            os.makedirs(structure_folder, exist_ok=True)

            # Save each slice of the 3D structure as a separate image
            for slice_idx in range(num_slices):
                slice_image = structure[slice_idx]

                # Save the slice as a binary BMP file
                slice_filename = os.path.join(structure_folder, f"slice_{slice_idx+1:03d}.bmp")
                plt.imsave(slice_filename, slice_image, cmap='gray', format='bmp', vmin=0, vmax=1)

            print(f"Saved 3D structure {structure_idx+1:05d} with {num_pores} pores (pore size {pore_size}x{pore_size}, {density_label}) in '{structure_folder}'.")

print(f"Generated all 3D porous material structures in '{output_folder}' folder.")


Saved 3D structure 00001 with 39 pores (pore size 10x10, dataset) in '3D_square/pore_10x10/dataset/structure_00000'.
Saved 3D structure 00002 with 32 pores (pore size 10x10, dataset) in '3D_square/pore_10x10/dataset/structure_00001'.
Saved 3D structure 00003 with 38 pores (pore size 10x10, dataset) in '3D_square/pore_10x10/dataset/structure_00002'.
Saved 3D structure 00004 with 12 pores (pore size 10x10, dataset) in '3D_square/pore_10x10/dataset/structure_00003'.
Saved 3D structure 00005 with 33 pores (pore size 10x10, dataset) in '3D_square/pore_10x10/dataset/structure_00004'.
Saved 3D structure 00006 with 39 pores (pore size 10x10, dataset) in '3D_square/pore_10x10/dataset/structure_00005'.
Saved 3D structure 00007 with 13 pores (pore size 10x10, dataset) in '3D_square/pore_10x10/dataset/structure_00006'.
Saved 3D structure 00008 with 15 pores (pore size 10x10, dataset) in '3D_square/pore_10x10/dataset/structure_00007'.
Saved 3D structure 00009 with 19 pores (pore size 10x10, dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

# Set the size of the 3D structure
slice_size = (100, 100)  # Each slice is 100x100 pixels
num_slices = 100         # Number of slices
num_structures_per_case = 200  # Number of structures for each case

# Pore sizes and their corresponding names
pore_sizes = [12]
density_labels = ['low_density', 'medium_density', 'high_density']

# Number of pores for each density
density_pores = {
    'low_density': (13, 15),    # Low density: 3-5 pores
    'medium_density': (16, 20), # Medium density: 6-10 pores
    'high_density': (21, 25)   # High density: 11-16 pores
}

# Directory to save the structures
output_folder = "3D_cirlinder"
os.makedirs(output_folder, exist_ok=True)

# Function to check if a circular pore overlaps with existing pores
def is_overlapping(existing_mask, center_row, center_col, radius):
    # Create a grid of points and check if any point within the circle overlaps
    y, x = np.ogrid[-radius:radius+1, -radius:radius+1]
    mask = x**2 + y**2 <= radius**2

    # Extract the area where the circle would be placed
    existing_area = existing_mask[max(0, center_row - radius):min(existing_mask.shape[0], center_row + radius + 1),
                                  max(0, center_col - radius):min(existing_mask.shape[1], center_col + radius + 1)]
    
    return np.any(existing_area[mask])

# Function to place multiple non-overlapping cylindrical pores in 3D structure
def generate_cylindrical_pores_3D(radius, num_pores):
    # Create a 3D array (num_slices x height x width) filled with zeros (black background)
    structure = np.zeros((num_slices, slice_size[0], slice_size[1]))

    for _ in range(num_pores):
        pore_placed = False
        while not pore_placed:
            # Randomly position the center of the cylindrical pore in the first slice
            center_row = np.random.randint(radius, slice_size[0] - radius)
            center_col = np.random.randint(radius, slice_size[1] - radius)

            # Check for overlap with existing pores in the first slice
            if not is_overlapping(structure[0], center_row, center_col, radius):
                # Create a mask for the cylindrical pore (same circle repeated in each slice)
                y, x = np.ogrid[-radius:radius+1, -radius:radius+1]
                mask = x**2 + y**2 <= radius**2

                # Place the cylindrical pore across all slices
                for slice_idx in range(num_slices):
                    structure[slice_idx, center_row - radius:center_row + radius + 1, center_col - radius:center_col + radius + 1][mask] = 1  # Pore is white (1)

                pore_placed = True

    return structure

# Generate and save 3D structures
for pore_size in pore_sizes:
    for density_label in density_labels:
        # Get the number of pores based on the density
        min_pores, max_pores = density_pores[density_label]

        # Create a folder for this pore size and density distribution
        case_folder = os.path.join(output_folder, f"pore_{pore_size}x{pore_size}", density_label)
        os.makedirs(case_folder, exist_ok=True)

        for structure_idx in range(num_structures_per_case):
            # Randomly generate the number of pores for this structure
            num_pores = np.random.randint(min_pores, max_pores + 1)

            # Generate the 3D structure with non-overlapping cylindrical pores
            radius = pore_size // 2
            structure = generate_cylindrical_pores_3D(radius, num_pores)

            # Create a folder for each 3D structure
            structure_folder = os.path.join(case_folder, f"structure_{structure_idx+1:03d}")
            os.makedirs(structure_folder, exist_ok=True)

            # Save each slice of the 3D structure as a separate image
            for slice_idx in range(num_slices):
                slice_image = structure[slice_idx]

                # Save the slice as a BMP file
                slice_filename = os.path.join(structure_folder, f"slice_{slice_idx+1:03d}.bmp")
                plt.imsave(slice_filename, slice_image, cmap='gray', format='bmp')

            print(f"Saved 3D structure {structure_idx+1:03d} with {num_pores} cylindrical pores (pore size {pore_size}, {density_label}) in '{structure_folder}'.")

print(f"Generated all 3D porous material structures with cylindrical pores in '{output_folder}' folder.")


The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.
Saved 3D structure 001 with 13 cylindrical pores (pore size 12, low_density) in '3D_cirlinder/pore_12x12/low_density/structure_001'.
Saved 3D structure 002 with 15 cylindrical pores (pore size 12, low_density) in '3D_cirlinder/pore_12x12/low_density/structure_002'.
Saved 3D structure 003 with 13 cylindrical pores (pore size 12, low_density) in '3D_cirlinder/pore_12x12/low_density/structure_003'.
Saved 3D structure 004 with 13 cylindrical pores (pore size 12, low_density) in '3D_cirlinder/pore_12x12/low_density/structure_004'.
Saved 3D structure 005 with 15 cylindrical pores (pore size 12, low_density) in '3D_cirlinder/pore_12x12/low_density/structure_005'.
Saved 3D structure 006 with 14 cylindrical pores (pore size 12, low_density) in '3D_cirlinder/pore_12x12/low_density/structure_006'.
Saved 3D structure 007 with 15 cylindrical pores (pore 